In [2]:
import itertools
import z3

# A Slice is a rectangular grid representing Life state at a specific time.
Slice = list[list[z3.ExprRef]]

def make_life(solver: z3.Solver, grid_size: int, time_steps: int) -> list[Slice]:
    """Create a (time steps) x (grid size) x (grid size) array of symbolic variables.

    Variables are named following the convention:
    "s_{t}_{i}_{j}", where t is the time step, i is the row, and j is the column.

    Args:
        solver (Solver): The Solver instance to add assertions and variables to.
        grid_size (int): The side-length of the grid to generate.
        time_steps (int): The number of time-steps to generate

    Returns:
        list[Slice]: _description_
    """
    # (tenzin): Note at some point we can potentially replace `Int`'s with `Bool`'s and instead use Z3
    # Pseudo-boolean constraints for counting boolean cardinality:
    # https://microsoft.github.io/z3guide/docs/logic/propositional-logic/#pseudo-boolean-constraints
    vars = [[[z3.Int(f's_{t}_{i}_{j}') for j in range(grid_size)] for i in range(grid_size)] for t in range(time_steps)]

    for t in range(0, time_steps):
        for i, j in itertools.product(range(grid_size), range(grid_size)):
            from z3 import Or, And, Implies
            # Each cell is either 0 or 1.
            solver.add(Or(vars[t][i][j] == 0, vars[t][i][j] == 1))

            if t == 0:
                continue

            # Life rules.
            count = 0
            for di, dj in itertools.product(range(-1, 2), range(-1, 2)):
                if di == dj == 0:
                    continue
                if 0 <= i + di < grid_size and 0 <= j + dj < grid_size:
                    count += vars[t - 1][i + di][j + dj]
            prev = vars[t - 1][i][j]
            next = vars[t    ][i][j]
            solver.add(Implies(And(prev == 1, next == 0), Or(count < 2, count > 3)))
            solver.add(Implies(And(prev == 1, next == 1), Or(count == 2, count == 3)))
            solver.add(Implies(And(prev == 0, next == 1), count == 3))
            solver.add(Implies(And(prev == 0, next == 0), count != 3))
    return vars

def print_model(model: z3.ModelRef, state: list[Slice]) -> None:
    """Pretty print the model for the given state."""
    for t, s in enumerate(state):
        print(f"t = {t}")
        for i in range(len(s)):
            for j in range(len(s[0])):
                print(model[state[t][i][j]], end=" ")
            print()
        print()

def constrain(solver: z3.Solver, s: Slice, on: set[tuple[int, int]]):
    """Constrain the given slice to be on at the given coordinates."""
    for i in range(len(s)):
        for j in range(len(s[0])):
            if (i, j) in on:
                solver.add(s[i][j] == 1)
            else:
                solver.add(s[i][j] == 0)


## Forward (sanity check)

In [3]:
# Sanity check: 2x2 grid stays still
solver = z3.Solver()
state = make_life(solver, grid_size=5, time_steps=4)
constrain(solver, state[0], set([(1, 1), (1, 2), (2, 1), (2, 2)]))
solver.check()
model = solver.model()
print_model(model, state)


t = 0
0 0 0 0 0 
0 1 1 0 0 
0 1 1 0 0 
0 0 0 0 0 
0 0 0 0 0 

t = 1
0 0 0 0 0 
0 1 1 0 0 
0 1 1 0 0 
0 0 0 0 0 
0 0 0 0 0 

t = 2
0 0 0 0 0 
0 1 1 0 0 
0 1 1 0 0 
0 0 0 0 0 
0 0 0 0 0 

t = 3
0 0 0 0 0 
0 1 1 0 0 
0 1 1 0 0 
0 0 0 0 0 
0 0 0 0 0 



In [3]:
# Sanity check: 1x3 rotation
solver = z3.Solver()
state = make_life(solver, grid_size=3, time_steps=4)
constrain(solver, state[0], set([(0, 1), (1, 1), (2, 1)]))
solver.check()
model = solver.model()
print_model(model, state)

t = 0
0 1 0 
0 1 0 
0 1 0 

t = 1
0 0 0 
1 1 1 
0 0 0 

t = 2
0 1 0 
0 1 0 
0 1 0 

t = 3
0 0 0 
1 1 1 
0 0 0 



In [4]:
# Sanity check: glider
solver = z3.Solver()
state = make_life(solver, grid_size=6, time_steps=6)
constrain(solver, state[0], set([(0, 1), (1, 2), (2, 0), (2, 1), (2, 2)]))
solver.check()
model = solver.model()
print_model(model, state)


t = 0
0 1 0 0 0 0 
0 0 1 0 0 0 
1 1 1 0 0 0 
0 0 0 0 0 0 
0 0 0 0 0 0 
0 0 0 0 0 0 

t = 1
0 0 0 0 0 0 
1 0 1 0 0 0 
0 1 1 0 0 0 
0 1 0 0 0 0 
0 0 0 0 0 0 
0 0 0 0 0 0 

t = 2
0 0 0 0 0 0 
0 0 1 0 0 0 
1 0 1 0 0 0 
0 1 1 0 0 0 
0 0 0 0 0 0 
0 0 0 0 0 0 

t = 3
0 0 0 0 0 0 
0 1 0 0 0 0 
0 0 1 1 0 0 
0 1 1 0 0 0 
0 0 0 0 0 0 
0 0 0 0 0 0 

t = 4
0 0 0 0 0 0 
0 0 1 0 0 0 
0 0 0 1 0 0 
0 1 1 1 0 0 
0 0 0 0 0 0 
0 0 0 0 0 0 

t = 5
0 0 0 0 0 0 
0 0 0 0 0 0 
0 1 0 1 0 0 
0 0 1 1 0 0 
0 0 1 0 0 0 
0 0 0 0 0 0 



## Edge Cases

In [9]:
# Edge case: Example of where forward can fail (Tenzin: this has implications
# for the results we get from predecessor queries too, i.e.: we can get
# incorrect sequences I'm pretty sure).
# This spinner on the bottom should spin indefinitely, so on t=2, it
# should go back to the bottom being set, however instead the solver
# predicts that we die out.
# One solution is to add a constraint on the imaginary boundary cells,
# asserting that they never come alive.
solver = z3.Solver()
state = make_life(solver, grid_size=3, time_steps=3)
constrain(solver, state[0], set([(2, 1), (2, 0), (2, 2)]))
solver.check()
model = solver.model()
print_model(model, state)

t = 0
0 0 0 
0 0 0 
1 1 1 

t = 1
0 0 0 
0 1 0 
0 1 0 

t = 2
0 0 0 
0 0 0 
0 0 0 



## Backwards

In [4]:
# Backwards: 3x3 corner
solver = z3.Solver()
state = make_life(solver, grid_size=5, time_steps=2)
constrain(solver, state[-1], set([(0,0)]))
solver.check()
model = solver.model()
print_model(model, state)

t = 0
1 1 0 1 0 
1 1 1 0 0 
0 1 0 0 0 
0 1 0 0 0 
0 0 0 0 0 

t = 1
1 0 0 0 0 
0 0 0 0 0 
0 0 0 0 0 
0 0 0 0 0 
0 0 0 0 0 



In [5]:
# Backwards: smiley
solver = z3.Solver()
state = make_life(solver, grid_size=6, time_steps=4)
constrain(solver, state[-1], set([(0,0)]))
solver.check()
model = solver.model()
print_model(model, state)


t = 0
0 1 0 1 0 1 
1 1 0 1 0 0 
0 0 0 1 1 0 
1 1 1 0 1 0 
0 0 1 1 1 1 
1 0 0 0 1 1 

t = 1
1 1 0 0 1 0 
1 1 0 1 0 0 
0 0 0 0 1 0 
0 1 0 0 0 0 
1 0 1 0 0 0 
0 0 0 0 0 1 

t = 2
1 1 1 0 0 0 
1 1 1 1 1 0 
1 1 1 0 0 0 
0 1 0 0 0 0 
0 1 0 0 0 0 
0 0 0 0 0 0 

t = 3
1 0 0 0 0 0 
0 0 0 0 0 0 
0 0 0 0 0 0 
0 0 0 0 0 0 
0 0 0 0 0 0 
0 0 0 0 0 0 



In [6]:
# Glider discovery
solver = z3.Solver()
state = make_life(solver, grid_size=6, time_steps=4)
for i in range(6):
    for j in range(6):
        solver.add(state[0][i][j] == state[-1][i][j])
solver.add(state[0][1][1] != state[1][1][1])
solver.check()
model = solver.model()
print_model(model, state)

t = 0
1 1 0 0 0 0 
1 1 0 0 1 0 
0 0 1 1 0 1 
0 0 1 0 0 1 
0 1 0 0 0 1 
0 0 1 1 1 0 

t = 1
1 1 0 0 0 0 
1 0 0 1 1 0 
0 0 1 1 0 1 
0 1 1 1 0 1 
0 1 0 0 0 1 
0 0 1 1 1 0 

t = 2
1 1 0 0 0 0 
1 0 0 1 1 0 
0 0 0 0 0 1 
0 1 0 1 0 1 
0 1 0 0 0 1 
0 0 1 1 1 0 

t = 3
1 1 0 0 0 0 
1 1 0 0 1 0 
0 0 1 1 0 1 
0 0 1 0 0 1 
0 1 0 0 0 1 
0 0 1 1 1 0 



In [7]:
# Backwards: big smiley
solver = z3.Solver()
n = 25
state = make_life(solver, grid_size=n, time_steps=3)
constrain(solver, state[-1], set([(5, 5), (5, 8), (7, 5), (7, 8), (8, 6), (8, 7)]))
# for t in range(3):
#     solver.add(sum([state[t][i][j] for j in range(n) for i in range(n)]) <= 20)
solver.check()
model = solver.model()
print_model(model, state)

t = 0
0 0 1 1 1 0 0 0 0 0 1 0 1 1 1 0 1 0 0 0 0 0 1 0 0 
1 1 1 0 0 0 0 1 0 0 0 0 1 1 1 1 0 0 1 1 0 0 1 0 0 
1 0 0 1 0 1 0 0 1 0 0 0 1 1 1 1 0 1 1 1 1 1 0 0 0 
1 1 0 0 0 0 0 1 1 1 1 0 1 0 0 0 0 1 0 1 1 1 1 0 1 
1 1 1 1 0 1 0 1 0 0 0 0 0 1 0 0 1 0 0 0 1 1 1 0 1 
0 0 1 0 0 0 1 0 1 0 1 0 0 1 1 0 1 0 0 1 0 0 1 0 1 
0 0 0 0 0 1 0 1 1 0 0 1 0 0 1 0 1 1 1 0 0 0 0 0 0 
0 1 1 0 1 0 0 0 0 1 0 0 0 1 0 0 0 1 0 0 1 1 0 0 0 
0 1 0 0 1 0 0 0 0 0 0 0 1 0 0 1 1 0 0 1 0 1 1 0 0 
1 0 0 1 1 1 1 0 1 0 1 1 0 0 1 0 1 0 0 1 0 0 0 0 1 
0 1 0 0 0 0 0 1 1 0 0 1 0 0 1 1 0 0 1 0 0 1 0 0 1 
1 0 1 0 0 0 0 0 0 1 0 1 1 1 0 0 0 0 1 0 0 1 0 1 1 
0 0 0 1 1 1 1 0 0 0 0 1 0 0 1 1 0 1 0 0 1 0 1 1 1 
0 0 1 0 0 1 1 1 0 0 1 1 0 1 0 1 0 1 0 0 1 1 1 1 1 
0 0 1 0 1 0 0 0 1 0 0 1 1 1 1 0 1 1 0 0 0 0 0 0 1 
1 0 0 0 0 1 1 1 1 1 0 0 0 0 0 0 0 0 0 1 0 0 0 0 0 
1 0 1 0 0 1 0 0 0 0 1 0 0 0 0 1 0 0 0 1 1 0 0 0 0 
0 0 0 0 1 0 0 0 0 0 1 0 0 1 1 0 0 0 0 1 0 1 0 0 0 
0 1 0 0 1 0 1 0 0 0 1 1 0 0 1 1 0 0 0 1 0 0 0 1 1 
1 1 0 1 0 0 1 1 0 0 0 1 0

## Garden of Eden

In [8]:
# Known GoE from wikipedia

pat = """
010110110010
001011101110
001101110101
010111011101
111110011110
011011101001
011101010010
011011100110
101110111010
100110010101
000000000010
000000000000
"""

solver = z3.Solver()
state = make_life(solver, grid_size = 14, time_steps = 2)
on = set([(i + 1, j + 1) for i, row in enumerate(pat.split()) for j, c in enumerate(row) if c == '1'])
constrain(solver, state[-1], on)
assert(solver.check() == z3.unsat)

NameError: name 'unsat' is not defined

In [ ]:
# Exhaustive search for Gardens of Eden
def all_grids(n):
    mask = 0
    while mask < 1 << (n * n):
        on = set([(i, j) for i in range(n) for j in range(n) if mask & (1 << (i * n + j))])
        yield on
        mask += 1

n = 3
for grid in all_grids(n):
    solver = z3.Solver()
    state = make_life(solver, grid_size=n+2, time_steps=2)
    offset_grid = set([(i + 1, j + 1) for i, j in grid])
    constrain(solver, state[-1], offset_grid)
    if solver.check() == z3.unsat:
        print(f"no solution for {grid}")

In [ ]:
# Incremental exhaustive search

def next(i, j):
    j += 1
    if j == n:
        i += 1
        j = 0
    return i, j

def remaining_grid(n, i, j):
    cells = []
    while i < n:
        cells.append((i, j))
        i, j = next(i, j)
    return cells

def traverse(n):
    yield []
    def visit(n, i, j):
        next_i, next_j = next(i, j)
        if next_i != n:
            yield from visit(n, next_i, next_j)

        if next_i == n:
            yield [(i, j, 1)]
        else:
            yield [(i, j, 1)] + list(map(lambda pos: pos + (0,), remaining_grid(n, next_i, next_j)))
            yield from visit(n, next_i, next_j)
    yield from visit(n, 0, 0)


n = 3
solver = z3.Solver()
state = make_life(solver, grid_size=n+2, time_steps=2)
end = state[-1]
for i, j in remaining_grid(n, 0, 0):
    solver.push()
    solver.add(end[i][j] == 0)

for updates in traverse(n):
    for _ in updates:
        solver.pop()
    for i, j, val in updates:
        solver.push()
        solver.add(end[i + 1][j + 1] == val)
    if solver.check() == unsat:
        print(f"no solution")

## Rule synthesis